In [3]:
!python -m tf2onnx.convert --tflite lightspeech_quant.tflite --output lightspeech_quant.onnx
!python -m tf2onnx.convert --tflite mbmelgan.tflite --output mbmelgan.onnx

2024-07-15 01:57:12.198244: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/s44504/miniconda3/envs/tftts2/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-07-15 01:57:12,735 - INFO - Using tensorflow=2.12.0, onnx=1.16.1, tf2onnx=1.16.1/15c810
2024-07-15 01:57:12,735 - INFO - Using opset <onnx, 15>
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-07-15 01:57:12,800 - WARNING - Replacing scalar output shape of length_regulator/Sum_1 with unknown shape
2024-07-15 01:57:12,801 - WARNING - Replacing scalar output shape of length_regulator/sub with unknown shape
2024-07-15 01:57:13,009 - INFO - Optimizing ONNX model
2024-07-15 01:57:14,227 - INFO - After optimization: Cast -100 (152->52), Concat -10 (34->24), Const -196 (432->236), Dequanti

2024-07-15 01:57:18,385 - INFO - After optimization: Cast -192 (218->26), Concat +4 (80->84), Const -542 (659->117), Gather +4 (9->13), Identity -108 (108->0), Reshape -80 (180->100), Split +4 (0->4), Squeeze -9 (83->74), Transpose -75 (183->108), Unsqueeze -66 (113->47)
2024-07-15 01:57:18,412 - INFO - 
2024-07-15 01:57:18,412 - INFO - Successfully converted TensorFlow model mbmelgan.tflite to ONNX
2024-07-15 01:57:18,412 - INFO - Model inputs: ['mels']
2024-07-15 01:57:18,412 - INFO - Model outputs: ['Identity']
2024-07-15 01:57:18,412 - INFO - ONNX model is saved at mbmelgan.onnx


In [4]:
from tensorflow_tts.inference import AutoProcessor

processor = AutoProcessor.from_pretrained("bookbot/lightspeech-mfa-sw-v2", use_auth_token=True)
processor.mode = "eval" # change processor from train to eval mode

In [5]:
import onnxruntime as ort

model_path = "./lightspeech_quant.onnx"
lightspeech = ort.InferenceSession(model_path)

model_path = "./mbmelgan.onnx"
mbmelgan = ort.InferenceSession(model_path)

In [6]:
text = "Kwa maana Mungu aliupenda ulimwengu kiasi cha kumtoa Mwanae pekee, ili kila mtu amwaminiye asipotee, bali awe na uzima wa milele."
input_ids = processor.text_to_sequence(text)

In [7]:
import numpy as np

input_ids = np.array([input_ids], dtype=np.int32)
speaker_ids = np.array([0], dtype=np.int32)
speed_ratios = np.array([1.0], dtype=np.float32)
f0_ratios = np.array([1.0], dtype=np.float32)
energy_ratios = np.array([1.0], dtype=np.float32)

In [8]:
# Run inference
inputs = {
    "input_ids": input_ids,
    "speaker_ids": speaker_ids,
    "speed_ratios": speed_ratios,
    "f0_ratios": f0_ratios,
    "energy_ratios": energy_ratios,
}

mel_output, durations, _ = lightspeech.run(None, inputs)
outputs = mbmelgan.run(None, {"mels": mel_output,})
audio_tflite = outputs[0][0, :, 0]

In [9]:
import soundfile as sf
sf.write("john3.wav", audio_tflite, 44100)

In [ ]:
mel_output.shape, audio_tflite.shape

In [ ]:
durations